In [1]:
from engine import TradingEngine

# 1. Preparar datos (usamos más datos para que el backtest sea significativo)
bot = TradingEngine()
df = bot.fetch_data(timeframe='1h', limit=500) 
df = bot.add_indicators(df)

# 2. Ejecutar Backtest
results = bot.run_backtest(df, initial_balance=1000)

print(f"--- Backtest Results for {bot.symbol} ---")
print(f"Start Balance: ${results['initial_balance']}")
print(f"End Balance:   ${results['final_balance']}")
print(f"Total Return:  {results['total_return_pct']}%")

# 3. Visualizar la curva de beneficios
results['df'][['cum_market_returns', 'cum_strategy_returns']].plot(figsize=(10, 6))

NameError: name 'rsi_window' is not defined

In [ ]:
import numpy as np
import mplfinance as mpf

# 1. Obtener datos procesados
df = bot.fetch_data(timeframe='1h', limit=150)
df = bot.add_indicators(df)
df = bot.get_signals(df)

# 2. Detectar cambios de señal (Crossovers)
# Usamos .diff() para ver dónde cambia la señal de un registro al siguiente
df['Signal_Change'] = df['Signal'].diff()

# Crear series para las flechas (rellenas con NaN donde no hay señal)
df['buy_price'] = np.where(df['Signal_Change'] > 0, df['close'] * 0.98, np.nan)
df['sell_price'] = np.where(df['Signal_Change'] < 0, df['close'] * 1.02, np.nan)

# 3. Preparar para graficar
df_plot = df.copy().set_index('timestamp')

# 4. Configurar las capas del gráfico (Addplots)
apds = [
    # Línea de la Media Móvil
    mpf.make_addplot(df_plot['SMA_20'], color='orange', width=1.5),
    
    # Flechas de Compra (Triángulos verdes hacia arriba)
    mpf.make_addplot(df_plot['buy_price'], type='scatter', markersize=100, marker='^', color='green'),
    
    # Flechas de Venta (Triángulos rojos hacia abajo)
    mpf.make_addplot(df_plot['sell_price'], type='scatter', markersize=100, marker='v', color='red')
]

# 5. Generar el gráfico
mpf.plot(df_plot, type='candle', style='charles',
         addplot=apds,
         title=f'BTC/USDT - Buy/Sell Signals',
         ylabel='Price (USDT)',
         figsize=(14, 8),
         volume=True)

In [ ]:
from engine import TradingEngine
import pandas as pd

bot = TradingEngine()

# Definimos los rangos que queremos probar
ventanas_a_probar = [5, 10, 20, 50, 100, 200]
resultados_optimizacion = []

# Descargamos los datos una sola vez para ser eficientes
df_base = bot.fetch_data(timeframe='1h', limit=1000)

for v in ventanas_a_probar:
    # Copiamos el dataframe para no ensuciar el original en cada iteración
    df_temp = df_base.copy()
    
    # Ejecutamos el flujo con la ventana actual
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    
    # Guardamos el resultado
    resultados_optimizacion.append({
        'Ventana': v,
        'Retorno %': res['total_return_pct'],
        'Final $': res['final_balance']
    })

# Convertimos a DataFrame para ver la comparativa
df_results = pd.DataFrame(resultados_optimizacion)
print(df_results.sort_values(by='Retorno %', ascending=False))

In [ ]:
import matplotlib.pyplot as plt

# 1. Definir rango amplio (de 5 a 300, saltando de 5 en 5)
ventanas = range(5, 301, 5)
retornos = []

df_base = bot.fetch_data(timeframe='1h', limit=2000) # Más datos = más fiabilidad

for v in ventanas:
    df_temp = df_base.copy()
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    retornos.append(res['total_return_pct'])

# 2. Graficar la curva de optimización
plt.figure(figsize=(12, 6))
plt.plot(ventanas, retornos, marker='o', linestyle='-', color='b', markersize=4)
plt.axhline(0, color='red', linestyle='--') # Línea de equilibrio (0% de beneficio)

plt.title('Optimización de Parámetros: Ventana SMA vs Rentabilidad')
plt.xlabel('Tamaño de la Ventana (SMA)')
plt.ylabel('Retorno Total (%)')
plt.grid(True, alpha=0.3)
plt.show()